In [31]:
from memgpt import create_client, Admin
from memgpt.client.client import LocalClient, RESTClient 

In [32]:
client = create_client() 
client.user_id

UUID('00000000-0000-0000-0000-000000000000')

In [33]:
client.list_models()

ListModelsResponse(models=[LLMConfigModel(model='gpt-4o-mini', model_endpoint_type='openai', model_endpoint='https://api.openai.com/v1', model_wrapper=None, context_window=8192)])

## Create a key from the Admin portal 
(This is to allow viewing agents on the dev portal) 

In [35]:
admin_client = Admin(base_url="http://localhost:8283", token="memgptadmin")

In [36]:
admin_client.create_key(user_id=client.user_id, key_name="key")

CreateAPIKeyResponse(api_key='sk-45cc3e1fd35a3fac3a2ad959fc23877a0476181e8b0a5557')

In [37]:
agent_state = client.create_agent()

MemGPT.memgpt.server.server - INFO - Created new agent from config: <memgpt.agent.Agent object at 0x14e542600>


## Create an agent 

In [38]:
def read_resume(self, name: str): 
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

def submit_candidate_for_outreach(self, candidate_name: str, resume: str, justification: str): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Summary reason for why the candidate is good and should be reached out to
    """
    from memgpt import create_client 
    client = create_client()
    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    # NOTE: we will define this agent later 
    #print("submit for outreach", message)
    response = client.send_message(agent_name="outreach_agent", role="user", message=message) # TODO: implement this
    #print(respose.messages)

# TODO: add an archival andidate tool (provide justification) 

read_resume_tool = client.create_tool(read_resume) 
submit_candidate_tool = client.create_tool(submit_candidate_for_outreach)

In [39]:
from memgpt.memory import ChatMemory

company_description = "The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents."
skills = "Front-end (React, Typescript), software engineering (ideally Python), and experience with LLMs."


leadgen_agent = client.create_agent(
    name="leadgen_agent", 
    memory=ChatMemory(
        persona=f"You are responsible to finding good recruiting candidates, for the company description: {company_description}. " \
        + f"Ideal canddiates have skills: {skills}. " \
        + "Search for candidates by calling the `search_candidates_db` function. " \
        + "When you find a good candidate, submit the candidate for outreach with the `submit_candidate_for_outreach` tool. " \
        + "Continue to search through the database until there are no more entries. ",
        human="",
    ), 
    tools=[read_resume_tool.name, submit_candidate_tool.name]
)

MemGPT.memgpt.server.server - INFO - Created new agent from config: <memgpt.agent.Agent object at 0x14e542600>


## Cleanup 

In [23]:
agents = client.list_agents()

In [25]:
for agent in agents: 
    client.delete_agent(agent.id)
    print("Deleted", agent.name)

Deleted FierceNucleus
Deleted LuxuriousRaccoon
